En particular los NaNs están expresados de varias maneras diferentes y habria que identificar si hay otras cosas que no estén estandarizadas y estandarizarlas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving set_de_datos_unificado_juzgado - set_de_datos_unificado.csv to set_de_datos_unificado_juzgado - set_de_datos_unificado.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['set_de_datos_unificado_juzgado - set_de_datos_unificado.csv']))


In [ ]:
df.head(10)

,N,NRO_REGISTRO,TOMO,FECHA_RESOLUCION,N_EXPTE_EJE,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,V_FISICA,V_PSIC,V_ECON,V_SEX,V_SOC,V_AMB,V_SIMB,FRASES_AGRESION,GENERO_ACUSADO/A,NACIONALIDAD_ACUSADO/A,EDAD_ACUSADO/A AL MOMENTO DEL HECHO,NIVEL_INSTRUCCION_ACUSADO/A,GENERO_DENUNCIANTE,NACIONALIDAD_DENUNCIANTE,EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO,NIVEL_INSTRUCCION_DENUNCIANTE,DOMICILIO_DENUNCIANTE,ASENTAMIENTO_O_VILLA,FRECUENCIA_EPISODIOS,RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE,HIJOS/AS_EN_COMUN,MEDIDAD DE PROTECCION VIGENTES AL MOMENTO DEL HECHO,ZONA_DEL_HECHO,LUGAR_DEL_HECHO\n,ASENTAMIENTO_O_VILLA.1,FECHA_DEL_HECHO,FECHA_DE_INICIO_DEL_HECHO,FECHA_DE_FINALIZACIÓN_DEL_HECHO,TIPO_DE_RESOLUCION,OBJETO_DE_LA_RESOLUCION,DETALLE,DECISION,ORAL_ESCRITA,HORA_DE_INICIO,HORA_DE_CIERRE,LINK,MODELO_DESTACADO,DURACION,SI/NO_RECURRENTE,DECISION_CAMARA_DE_APELACIONES,N_REGISTRO_Y_TOMO_CAMARA,LINK_CAMARA,SI/NO_RECURRENTE.1,DECISION_DE_ADMISIBILIDAD_CAMARA,N_REGISTRO_Y_TOMO_CAMARA_1,LINK_CAMARA_1,QUEJA_Y_RECURRENTE,DECISION_DE_ADMISIBILIDAD_TSJ,Nº_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN
0,1.0,NaN,NaN,1_8_2016,9498_16,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,NaN,usuario_de_facebook,argentino,no_corresponde,s/d,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,s/d,s/d,no,27_03_16,NaN,NaN,definitiva,prision_preventiva,conocimiento_personal,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,1_8_2016,9904_16,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,no,si,no,no,no,no,si,me la vas a pagar,masculino,s/d,17,s/d,femenino,argentina,17,secundario_en_curso,zona_oeste,no,primera_vez,ex_pareja,no,no,zona_oeste,en_domicilio_particular,no,25_3_16,NaN,NaN,definitiva,archivo,convalida,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,2_8_2016,1313_16,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,si,si,si,no,no,no,si,NaN,masculino,argentino,50,terciario_incompleto,femenino,argentina,23,s/d,zona_oeste,no,esporadico,ex_pareja,si,no,zona_oeste,en_domicilio_particular,no,24_7_14,NaN,NaN,interlocutoria,medida_cautelar,rebeldia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,2_8_2016,2894_16,Pablo_Casas,contravencional,73,codigo_contravencional,incumplir_clausura,NaN,no,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,NaN,persona_juridica,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,zona_sur,en_domicilio_laboral,no,03_06_15,NaN,NaN,definitiva,suspension_del_proceso_a_prueba,extincion_accion_sobreseimiento,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,2_8_2016,3024_14,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,no,si,si,no,si,no,si,NaN,masculino,s/d,34,s/d,femenino,s/d,s/d,primario_incompleto,zona_oeste,no,s/d,s/d,s/d,s/d,zona_oeste,s/d,no,5_01_14,NaN,NaN,interlocutoria,suspension_del_proceso_a_prueba,admisibilidad,hace_lugar,oral,13:40,13:55,https://drive.goog

In [ ]:
# Solo trabajo con Violencia de genero 
df_violencia = df[df["VIOLENCIA_DE_GENERO"] == "si"].copy()
df_violencia.reset_index(drop=True, inplace=True)
df_violencia.shape

(891, 69)

In [ ]:
print (df_violencia.isnull().sum())


N                                      209
NRO_REGISTRO                            29
TOMO                                    29
FECHA_RESOLUCION                         0
N_EXPTE_EJE                              0
                                      ... 
LINK_TSJ                               891
RECURSO_EXTRAORDINARIO_Y_RECURRENTE    891
DECISION_CSJN                          891
N_REGISTRO_Y_TOMO_CSJN                 891
LINK_CSJN                              891
Length: 69, dtype: int64


In [ ]:
df_violencia.replace('', np.nan, inplace=True)

In [ ]:
df_violencia.drop(["MODELO_DESTACADO", "LINK_TSJ", 'RECURSO_EXTRAORDINARIO_Y_RECURRENTE', 'DECISION_CSJN', 'N_REGISTRO_Y_TOMO_CSJN','LINK_CSJN','N_REGISTRO_Y_TOMO_TSJ'], axis=1, inplace=True)


In [ ]:
df_violencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 62 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   N                                                    682 non-null    float64
 1   NRO_REGISTRO                                         862 non-null    object 
 2   TOMO                                                 862 non-null    float64
 3   FECHA_RESOLUCION                                     891 non-null    object 
 4   N_EXPTE_EJE                                          891 non-null    object 
 5   FIRMA                                                891 non-null    object 
 6   MATERIA                                              891 non-null    object 
 7   ART_INFRINGIDO                                       891 non-null    object 
 8   CODIGO_O_LEY                                         891 non-null    o

In [ ]:
# forward-fill
df_violencia.N.fillna(method='ffill', inplace= True)
#entendemos que los que estan en blanco es porque refieren al mismo expediente que arriba pero respecto de delitos distintos. 
df_violencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 62 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   N                                                    891 non-null    float64
 1   NRO_REGISTRO                                         862 non-null    object 
 2   TOMO                                                 862 non-null    float64
 3   FECHA_RESOLUCION                                     891 non-null    object 
 4   N_EXPTE_EJE                                          891 non-null    object 
 5   FIRMA                                                891 non-null    object 
 6   MATERIA                                              891 non-null    object 
 7   ART_INFRINGIDO                                       891 non-null    object 
 8   CODIGO_O_LEY                                         891 non-null    o